# Solution idea is from
#### Original (but not a notebook)
https://pclubiiti.github.io/blog/Inroduction-toSAT-solving/

Uses http://binarypuzzle.com

### remark: not Python 3 used

# completed by Peter Gragert

# a Binary Puzzel is a 2n x 2n quadratic scheme
consisting of 0s and 1s and .s (denoting not yet known) boxes.


# The rules

Each binary puzzle should be solved according to the following rules:

- Each box should contains a zero or a one.
- No more than two similar numbers next to or below each other are allowed.
- Each row and each column should contain an equal number of zeros and ones.
- Each row is unique and each column is unique. 

<p style="font-family:Courier; color:Blue; font-size: 20px;">
A really good bindary puzzel should have ONE solution and can be solved without gessing.</p>

In [1]:
# example of a 10 x 10 puzzel (ONE solution)

game = """
...00....1
........01
..0...1.0.
.1...1....
.0.0.11...
1.....1...
.0...1....
...1.1.11.
..........
..00......
"""

from z3 import *
import numpy as np # to show (first) solution matrice
s = Solver()
vals = []


def SolvePuzzel(game, maxCounter = 2):
    """
    game is to be solved
    default counter maximized by 2 to see if
    there is more than ONE solution
    solutions are stored in a dictionary and returned
    
    PKHG>HOW????
    
    """
    global vals
    #for controlling the number of solutions!
    result = dict()
    counter = 0 #while loop uses it
    
    #check input game!
    try:
        game = [y for y in filter(lambda x:x in '01.', game)]
        sizeXsize = len(game)
        size = int(math.sqrt(sizeXsize))
        assert(size * size == sizeXsize) 
        half_int_size = int(size/2)
        assert(size == 2 * int(size/2)) #even number of rows and columns
    except:
        return("check input game")
    #  a list cordinates to access (later) rows or columns)
    vs = [(r,c) for r in range(size) for c in range(size)]

    #  all cells declared as BitVectors Z3-variables
    # _ used to   rows and colum indices
    bV  = [[z3.BitVec('v_%d_%d' % (r,c), 5)  for c in range(size)] 
      for r in range(size)]
    
    #initializing all variables using the given info
    # either 0 or 1 or condition BINARY (value 0 or 1)
    
    # the z3-solver
    #s = Solver()

    for ((r,c), val) in zip(vs, game):
        if val == '.':  s.add(0 <= bV[r][c], bV[r][c] <= 1)
        else:           s.add(bV[r][c] == (int(val)))

    #helpers for row and colum indices 
    def row(n):   return [(n, i) for i in range(size)]
    def col(n):   return [(i, n) for i in range(size)]

    #Puzzel condition: rows, columns must sum up to size/2
    for i in range(size):
        s.add(z3.Sum([bV[r][c] for r,c in row(i)]) == (size/2))
        s.add(z3.Sum([bV[r][c] for r,c in col(i)]) == (size/2))

    # puzzel condition at moost two 1s or 0s in row resp. column
    # help functions for 3 consecutive cells (rows or cols)
    def row3(n,m):   return [(n, i) for i in range(m,m+3)]
    def col3(n,m):   return [(i, n) for i in range(m,m+3)]
    
    for i in range(size):
        for j in range(size-2):
            #at most two 1s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) <= 2)
            #at most two 0s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) >= 1)
            # same for the columns
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) <= 2)
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) >= 1)

    
    #That just leaves Rule #4 Each row is unique 
    # and each column is unique. Z3 has a Unique keyword.
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in row(i)])
                       for i in range(size)]))
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in col(i)]) 
                       for i in range(size)]))

    #result = s.check()
    while s.check() == sat and counter < maxCounter:
        m = s.model()
        result[counter] = s.model()
        counter += 1
        sol = [[m[bV[r][c]] for c in range(size)] for r in range(size)]
        print_matrix(sol)
        #pp(sol)
        if counter >= 1:
        #PKHG>want to save first resp. all solutions
            'todo'
        # deny found solution
        vals = [bV[i][j] == m.evaluate(bV[i][j]) for j in range(size)
            for i in range(size)]
        #PKHG>????
        '''
        for i in range(2):
            print(i, vals[i])
        '''
        s.add(Not(And(vals)))
    if counter == 1:
        print("Exactly ONE solution")
    elif counter > 1:
        print("several solutions found:" , counter)
    else:
        print("no solution")
    return result



In [2]:
g12 = """
.......00.1.
..1.......1.
...0.0..0..0
.....0......
.11.1..1.0..
.11.0.00....
10.1........
1..00...0...
............
..1.....1...
...0..0..0.0
..1...0.....
"""
res = SolvePuzzel(g12)

[[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
 [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1],
 [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
 [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
 [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
 [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]]
[[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
 [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
 [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]]
several solutions found: 2


In [3]:
len(res.keys())

2

### stop here ??

In [4]:
#PKHG>??? and now???? rr

In [5]:
s = Solver()
g2 = """
..
..
"""
resg2 = SolvePuzzel(g2)

[[0, 1], [1, 0]]
[[1, 0], [0, 1]]
several solutions found: 2


In [6]:
resg2

{0: [v_1_0 = 1, v_1_1 = 0, v_0_0 = 0, v_0_1 = 1],
 1: [v_0_1 = 0, v_0_0 = 1, v_1_1 = 1, v_1_0 = 0]}

In [7]:
r0 = resg2[0]
type(r0), r0

(z3.z3.ModelRef, [v_1_0 = 1, v_1_1 = 0, v_0_0 = 0, v_0_1 = 1])

In [8]:
type(r0)

z3.z3.ModelRef

In [9]:
#could be used ??? HOW???
r0.sexpr()

'(define-fun v_1_0 () (_ BitVec 5)\n  #b00001)\n(define-fun v_1_1 () (_ BitVec 5)\n  #b00000)\n(define-fun v_0_0 () (_ BitVec 5)\n  #b00000)\n(define-fun v_0_1 () (_ BitVec 5)\n  #b00001)'

In [10]:
r0

[v_1_0 = 1, v_1_1 = 0, v_0_0 = 0, v_0_1 = 1]

In [11]:
s = Solver()
g4 = """
0...
....
1.1.
..10
"""
SolvePuzzel(g4,100)

[[0, 1, 0, 1], [1, 0, 0, 1], [1, 0, 1, 0], [0, 1, 1, 0]]
Exactly ONE solution


{0: [v_1_0 = 1,
  v_2_3 = 0,
  v_0_2 = 0,
  v_3_1 = 1,
  v_3_0 = 0,
  v_1_2 = 0,
  v_1_3 = 1,
  v_2_1 = 0,
  v_0_3 = 1,
  v_1_1 = 0,
  v_0_1 = 1,
  v_3_3 = 0,
  v_3_2 = 1,
  v_2_2 = 1,
  v_2_0 = 1,
  v_0_0 = 0]}

In [12]:
#import numpy as np
myg4 = np.matrix([[0, 1, 0, 1], [1, 0, 0, 1], [1, 0, 1, 0], [0, 1, 1, 0]])

In [13]:
myg4

matrix([[0, 1, 0, 1],
        [1, 0, 0, 1],
        [1, 0, 1, 0],
        [0, 1, 1, 0]])

#  12x12 puzzel but with more then 100 solutions
### computed above (186 on my PC)

In [14]:
r0t = [res[0][i] for i in range(144)]
r0t

[v_3_2,
 v_8_8,
 v_7_2,
 v_9_7,
 v_8_3,
 v_2_7,
 v_5_10,
 v_2_10,
 v_5_9,
 v_3_9,
 v_11_5,
 v_2_9,
 v_9_0,
 v_9_5,
 v_8_11,
 v_2_1,
 v_1_6,
 v_0_3,
 v_0_1,
 v_2_6,
 v_11_0,
 v_6_9,
 v_5_5,
 v_6_11,
 v_11_7,
 v_3_7,
 v_9_10,
 v_1_8,
 v_6_8,
 v_9_9,
 v_10_1,
 v_8_10,
 v_1_7,
 v_1_11,
 v_0_5,
 v_4_10,
 v_8_0,
 v_1_0,
 v_10_5,
 v_8_4,
 v_11_10,
 v_10_2,
 v_4_5,
 v_0_9,
 v_3_3,
 v_3_6,
 v_7_7,
 v_10_8,
 v_1_1,
 v_8_2,
 v_3_4,
 v_0_2,
 v_3_0,
 v_1_9,
 v_7_5,
 v_9_3,
 v_4_3,
 v_0_0,
 v_11_11,
 v_9_11,
 v_4_6,
 v_2_4,
 v_8_1,
 v_6_7,
 v_4_8,
 v_8_5,
 v_7_6,
 v_6_4,
 v_3_1,
 v_8_9,
 v_8_7,
 v_8_6,
 v_10_4,
 v_6_10,
 v_11_9,
 v_5_11,
 v_5_8,
 v_11_1,
 v_9_6,
 v_2_0,
 v_1_5,
 v_7_11,
 v_4_0,
 v_10_7,
 v_11_8,
 v_9_1,
 v_6_6,
 v_2_2,
 v_0_11,
 v_11_3,
 v_10_0,
 v_0_4,
 v_7_10,
 v_0_6,
 v_4_11,
 v_10_10,
 v_7_1,
 v_7_9,
 v_11_4,
 v_9_4,
 v_6_5,
 v_5_3,
 v_5_0,
 v_3_11,
 v_1_4,
 v_1_3,
 v_6_2,
 v_3_10,
 v_3_8,
 v_11_6,
 v_11_2,
 v_10_11,
 v_10_9,
 v_10_6,
 v_10_3,
 v_9_8,
 v_9_2,
 v_7_8,
 v_7_4,
 v_

In [15]:
#r0t[143] ==> v_0_7

In [16]:
r0n = [(el, res[0][el]) for el in r0t]
len(r0n), r0n

(144,
 [(v_3_2, 0),
  (v_8_8, 1),
  (v_7_2, 1),
  (v_9_7, 0),
  (v_8_3, 1),
  (v_2_7, 1),
  (v_5_10, 0),
  (v_2_10, 0),
  (v_5_9, 1),
  (v_3_9, 0),
  (v_11_5, 1),
  (v_2_9, 1),
  (v_9_0, 1),
  (v_9_5, 0),
  (v_8_11, 1),
  (v_2_1, 1),
  (v_1_6, 0),
  (v_0_3, 1),
  (v_0_1, 1),
  (v_2_6, 1),
  (v_11_0, 0),
  (v_6_9, 0),
  (v_5_5, 1),
  (v_6_11, 1),
  (v_11_7, 1),
  (v_3_7, 0),
  (v_9_10, 0),
  (v_1_8, 1),
  (v_6_8, 1),
  (v_9_9, 1),
  (v_10_1, 1),
  (v_8_10, 0),
  (v_1_7, 1),
  (v_1_11, 1),
  (v_0_5, 0),
  (v_4_10, 1),
  (v_8_0, 0),
  (v_1_0, 0),
  (v_10_5, 1),
  (v_8_4, 1),
  (v_11_10, 0),
  (v_10_2, 0),
  (v_4_5, 1),
  (v_0_9, 1),
  (v_3_3, 1),
  (v_3_6, 1),
  (v_7_7, 1),
  (v_10_8, 0),
  (v_1_1, 0),
  (v_8_2, 0),
  (v_3_4, 0),
  (v_0_2, 0),
  (v_3_0, 1),
  (v_1_9, 0),
  (v_7_5, 1),
  (v_9_3, 1),
  (v_4_3, 0),
  (v_0_0, 0),
  (v_11_11, 1),
  (v_9_11, 0),
  (v_4_6, 0),
  (v_2_4, 1),
  (v_8_1, 1),
  (v_6_7, 0),
  (v_4_8, 0),
  (v_8_5, 0),
  (v_7_6, 0),
  (v_6_4, 1),
  (v_3_1, 0),
  (v_8_9

In [17]:
type(r0n), len(r0n)

(list, 144)

In [18]:
def convert_v_in_coord(name):
    tmp = str(name)
    tmp2 = tmp.split('_')
    #print(tmp2)
    tmp3 = eval(tmp2[1])
    tmp4 = tmp2[2][:-1]
    #print("tmp4" , tmp4, type(tmp4))
    tmp5 = tmp4.split(',')
    return([tmp3, eval(tmp5[0])],eval(tmp5[1]) )
    
convert_v_in_coord(r0n[0])


([3, 2], 0)

In [19]:
results = [convert_v_in_coord(el) for el in r0n]
len(results),results

(144,
 [([3, 2], 0),
  ([8, 8], 1),
  ([7, 2], 1),
  ([9, 7], 0),
  ([8, 3], 1),
  ([2, 7], 1),
  ([5, 10], 0),
  ([2, 10], 0),
  ([5, 9], 1),
  ([3, 9], 0),
  ([11, 5], 1),
  ([2, 9], 1),
  ([9, 0], 1),
  ([9, 5], 0),
  ([8, 11], 1),
  ([2, 1], 1),
  ([1, 6], 0),
  ([0, 3], 1),
  ([0, 1], 1),
  ([2, 6], 1),
  ([11, 0], 0),
  ([6, 9], 0),
  ([5, 5], 1),
  ([6, 11], 1),
  ([11, 7], 1),
  ([3, 7], 0),
  ([9, 10], 0),
  ([1, 8], 1),
  ([6, 8], 1),
  ([9, 9], 1),
  ([10, 1], 1),
  ([8, 10], 0),
  ([1, 7], 1),
  ([1, 11], 1),
  ([0, 5], 0),
  ([4, 10], 1),
  ([8, 0], 0),
  ([1, 0], 0),
  ([10, 5], 1),
  ([8, 4], 1),
  ([11, 10], 0),
  ([10, 2], 0),
  ([4, 5], 1),
  ([0, 9], 1),
  ([3, 3], 1),
  ([3, 6], 1),
  ([7, 7], 1),
  ([10, 8], 0),
  ([1, 1], 0),
  ([8, 2], 0),
  ([3, 4], 0),
  ([0, 2], 0),
  ([3, 0], 1),
  ([1, 9], 0),
  ([7, 5], 1),
  ([9, 3], 1),
  ([4, 3], 0),
  ([0, 0], 0),
  ([11, 11], 1),
  ([9, 11], 0),
  ([4, 6], 0),
  ([2, 4], 1),
  ([8, 1], 1),
  ([6, 7], 0),
  ([4, 8], 0),

In [20]:
#convert_v_in_coord(r0n[1])[1:]

In [21]:
r0n[:3]

[(v_3_2, 0), (v_8_8, 1), (v_7_2, 1)]

In [22]:
m = np.zeros((12,12), dtype = int)
#m

In [23]:
for el in results:
    if el[1] == 1:
        #print(m[el[0][0], el[0][1]])
        m[el[0][0], el[0][1]] = 1
m

array([[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

In [24]:
for i in range(12):
    print(sum(m[i]))

6
6
6
6
6
6
6
6
6
6
6
6


In [25]:
m.T

array([[0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1],
       [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1]])

In [26]:
mT = m.T
for i in range(12):
    print(sum(mT[i]))

6
6
6
6
6
6
6
6
6
6
6
6


In [27]:
'finish so far'

'finish so far'